Import

In [215]:
import pandas as pd
import re

Read CSV

In [216]:
df_bordeaux = pd.read_csv("CSV/bordeaux/listings_bordeaux.csv")


In [217]:
df_bordeaux.columns


Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'name',
       'description', 'neighborhood_overview', 'picture_url', 'host_id',
       'host_url', 'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'ca

### Question n°1  

In [218]:
df_bordeaux.groupby("neighbourhood_cleansed").apply(lambda s: pd.Series({ 
    "host count": s["host_id"].nunique(), 
    "sum reviews": s["number_of_reviews"].sum(), 
}))


,host count,sum reviews
neighbourhood_cleansed,,
3M-Bourgailh,8,98
Ambars-et-Lagrave,65,904
Ambs,49,1714
Arago-La Chataigneraie,43,1583
Arlac,90,2973
...,...,...
Sardine,17,788
Talence,429,13571
Toctoucau,16,479


### Question n°2

In [219]:
df_host_acceptance_rate=df_bordeaux["host_acceptance_rate"].str.rstrip('%').astype(float)

df_host_acceptance_rate.mean()

82.1015107341638

In [220]:
df_host_response_rate=df_bordeaux["host_response_rate"].str.rstrip('%').astype(float)

df_host_response_rate.mean()

93.4806598407281

### Question n°3

In [221]:
df_bordeaux[['id',"host_verifications"]].groupby("host_verifications").count()/len(df_bordeaux)*100

,id
host_verifications,
"['email', 'phone', 'work_email']",6.219568
"['email', 'phone']",85.319247
"['email', 'work_email']",0.009187
['email'],0.413413
"['phone', 'work_email']",0.220487
['phone'],7.533303
[],0.284796


In [222]:
df_bordeaux.replace(to_replace="work_email", value='work', regex=True,inplace=True)

In [223]:
email = df_bordeaux["host_verifications"].apply(lambda elt: 1 if 'email' in elt else 0 ).sum()
phone = df_bordeaux["host_verifications"].apply(lambda elt: 1 if 'phone' in elt else 0 ).sum()
work = df_bordeaux["host_verifications"].apply(lambda elt: 1 if 'work' in elt else 0 ).sum()
vide = df_bordeaux["host_verifications"].apply(lambda elt: 1 if elt == "[]" else 0 ).sum()
  
print("Il y a {0} personnes qui utilise un email pour la vérification soit {1} %".format(email,email/len(df_bordeaux)*100))
print("Il y a {0} personnes qui utilise un téléphone pour la vérification soit {1} %".format(phone,phone/len(df_bordeaux)*100))
print("Il y a {0} personnes qui utilise un email pro pour la vérification soit {1} %".format(work,work/len(df_bordeaux)*100))
print("Il y a {0} personnes qui n'ont pas  pour la vérification soit {1} %".format(vide,vide/len(df_bordeaux)*100))

Il y a 10010 personnes qui utilise un email pour la vérification soit 91.96141479099678 %
Il y a 10808 personnes qui utilise un téléphone pour la vérification soit 99.29260450160771 %
Il y a 702 personnes qui utilise un email pro pour la vérification soit 6.449242076251722 %
Il y a 31 personnes qui n'ont pas  pour la vérification soit 0.2847955902618282 %


### Question n°4

In [224]:
df_bordeaux["np_amenities"] =  df_bordeaux['amenities'].apply(lambda elt: re.sub("\[|\]|\"","",elt))
df_bordeaux["np_amenities"] = df_bordeaux["np_amenities"].apply(lambda elt: elt.split(","))


In [225]:
df_bordeaux["np_amenities"] =  df_bordeaux["np_amenities"].apply(lambda x: len(x))


In [226]:
df_bordeaux[["np_amenities","room_type"]].groupby('room_type').agg(["mean","std"])

np_amenities           
                        mean        std
room_type                              
Entire home/apt    27.603407  14.178751
Hotel room         26.177778  14.458858
Private room       21.365678  12.438094
Shared room        15.411765   7.957415

### Question n° 5

In [227]:
df_bordeaux["price"]= df_bordeaux["price"].str.replace(',',"")
df_bordeaux["price"]= df_bordeaux["price"].str.replace('$',"").astype(float)

/tmp/ipykernel_25537/484732451.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_bordeaux["price"]= df_bordeaux["price"].str.replace('$',"").astype(float)


In [228]:
def q1(x):
    return x.quantile(0.25)
def q3(x):
    return x.quantile(0.75)

In [229]:
df_bordeaux[["price","room_type"]].groupby('room_type').agg(["median","max","min",q1,q3])

price                              
                median     max   min      q1      q3
room_type                                           
Entire home/apt   84.0  8890.0  12.0   60.00  130.00
Hotel room       131.0   391.0   0.0  100.00  169.00
Private room      40.0  1492.0  10.0   30.00   60.00
Shared room       42.5   275.0   9.0   25.75   49.25

### Question n°6

In [230]:
df_bordeaux["bathrooms_text"].unique()

array(['1.5 baths', '3 baths', '1 bath', '1 private bath', nan, '2 baths',
       '1 shared bath', 'Shared half-bath', '4 baths', '8 baths',
       '5 baths', '4.5 baths', '2.5 baths', '0 baths', '1.5 shared baths',
       '3.5 baths', 'Half-bath', '3 shared baths', '6 baths', '5.5 baths',
       '7 baths', '0 shared baths', '2.5 shared baths',
       'Private half-bath', '2 shared baths', '6.5 baths',
       '4 shared baths', '11.5 baths', '7.5 baths'], dtype=object)

In [231]:
def first(s):
    return(re.findall("\d+\.\d+|\d+", s))

def last(s):
    return(re.split('(\d+)', s)[-1])

In [232]:
df_bordeaux['number_bath'] = df_bordeaux['bathrooms_text'].apply(lambda x :  first(str(x)))
df_bordeaux['bath_describe'] = df_bordeaux['bathrooms_text'].apply(lambda x : last(str(x)))
df_bordeaux['number_bath'] = pd.DataFrame([x for x in df_bordeaux['number_bath'] ]) # remove list from N

In [233]:
mask = df_bordeaux["bath_describe"] == "nan"
df_bordeaux[mask]
df_bordeaux_clean = df_bordeaux.drop(df_bordeaux[mask].index)

In [234]:
df_bordeaux_clean['number_bath'].unique()

array(['1.5', '3', '1', '2', None, '4', '8', '5', '4.5', '2.5', '0',
       '3.5', '6', '5.5', '7', '6.5', '11.5', '7.5'], dtype=object)

In [235]:
df_bordeaux_clean["bath_describe"].unique()

array([' baths', ' bath', ' private bath', ' shared bath',
       'Shared half-bath', ' shared baths', 'Half-bath',
       'Private half-bath'], dtype=object)

In [236]:
df_bordeaux_clean['number_bath'] =df_bordeaux_clean['number_bath'].apply(lambda x: 1 if (x is None) else float(x))

In [237]:
df_bordeaux_clean["bath_describe"]=df_bordeaux_clean["bath_describe"].str.lower()

In [238]:
df_bordeaux_clean["bath_describe"].unique()

array([' baths', ' bath', ' private bath', ' shared bath',
       'shared half-bath', ' shared baths', 'half-bath',
       'private half-bath'], dtype=object)

In [239]:

df_bordeaux_clean['private'] = df_bordeaux_clean['bath_describe'].apply(lambda x: 2 if ("private" in x) else 1)
df_bordeaux_clean['shared'] = df_bordeaux_clean['bath_describe'].apply(lambda x: 0.5 if ("shared" in x) else 1)
df_bordeaux_clean['half_bath'] = df_bordeaux_clean['bath_describe'].apply(lambda x: 0.5 if ("shared" in x) else 0)



In [240]:
df_bordeaux_clean['number_bath'] = (df_bordeaux_clean['number_bath'] + df_bordeaux_clean['half_bath']) * df_bordeaux_clean['private']*df_bordeaux_clean['shared']

In [241]:
df_bordeaux_clean["number_bath"].unique()

array([ 1.5 ,  3.  ,  1.  ,  2.  ,  0.75,  4.  ,  8.  ,  5.  ,  4.5 ,
        2.5 ,  0.  ,  3.5 ,  1.75,  6.  ,  5.5 ,  7.  ,  0.25,  1.25,
        6.5 ,  2.25, 11.5 ,  7.5 ])

In [242]:
df_bordeaux_clean[["host_id","number_bath"]].groupby("number_bath").count()

,host_id
number_bath,
0.00,34
0.25,14
0.75,1154
1.00,6494
1.25,38
1.50,486
1.75,7
2.00,2128
2.25,2


### Question n°7

In [243]:
df_bordeaux_simple = df_bordeaux[["number_of_reviews",'description']]

In [244]:
df_bordeaux_simple["len_description"] = df_bordeaux_simple['description'].apply(lambda x: len(str(x)))

/tmp/ipykernel_25537/1551447806.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bordeaux_simple["len_description"] = df_bordeaux_simple['description'].apply(lambda x: len(str(x)))


In [245]:
df_bordeaux_simple["len_description"]

0        1000
1         912
2        1000
3        1000
4        1000
         ... 
10880    1000
10881    1000
10882     809
10883    1000
10884    1000
Name: len_description, Length: 10885, dtype: int64

In [246]:
df_bordeaux_simple["number_of_reviews"].corr(df_bordeaux_simple['len_description'])

0.1887451422744717

### question n°8

In [247]:
df_reviews_bordeaux = pd.read_csv("CSV/bordeaux/reviews_bordeaux.csv")

In [248]:
df_reviews_bordeaux.columns

Index(['Unnamed: 0', 'listing_id', 'id', 'date', 'reviewer_id',
       'reviewer_name'],
      dtype='object')

In [249]:
df_reviews_bordeaux.columns=['Unnamed: 0', 'listing_id', 'id_review', 'date', 'reviewer_id',
       'reviewer_name']

In [250]:
df_reviews_bordeaux= df_reviews_bordeaux[['listing_id', 'id_review', 'date', 'reviewer_id',
       'reviewer_name']]

In [251]:
df_reviews_bordeaux.columns

Index(['listing_id', 'id_review', 'date', 'reviewer_id', 'reviewer_name'], dtype='object')

In [252]:
df_bordeaux.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'name',
       'description', 'neighborhood_overview', 'picture_url', 'host_id',
       'host_url', 'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'ca

In [253]:
df_merge = df_reviews_bordeaux.merge(df_bordeaux, how='left',left_on='listing_id',right_on='id') 

In [254]:
df_merge[["host_id","host_name","reviewer_id","reviewer_name"]]

,host_id,host_name,reviewer_id,reviewer_name
0,1127981,Luis,610681,Dallas
1,1127981,Luis,2005513,Nicolas
2,1127981,Luis,1937156,Robert
3,1127981,Luis,2001261,He
4,1127981,Luis,1766812,Etienne
...,...,...,...,...
306351,148912155,Valentin,40877952,Jude
306352,478090822,Mohammad,151433357,Moussa
306353,478090822,Mohammad,422815067,Lucas
306354,347361887,Matthieu,270567347,Belgin


In [255]:
mask = df_merge["host_name"] ==df_merge["reviewer_name"]


In [256]:
df_merge[["host_name","reviewer_name"]][mask]

,host_name,reviewer_name
490,Audrey,Audrey
1741,Marion,Marion
2454,Audrey,Audrey
3200,Christophe,Christophe
3702,Christophe,Christophe
...,...,...
303048,Sandrine,Sandrine
304817,Caroline,Caroline
304911,Stéphanie,Stéphanie
305259,Marine,Marine


In [257]:
len(df_merge[["host_name","reviewer_name"]][mask])/len(df_merge)*100

0.18475237958453564